In [3]:
import os
from tqdm import tqdm
import csv
import random
import llm_client
from AudiosetDataset import AudiosetDataset
import torch
import json

print(os.chdir("/home/junchenl/diffusers_with_dataloader/examples/text_to_image/audio_gen_files"))

NameError: name '_C' is not defined

In [ ]:
local = "/home/junchenl/AudioTaggingDoneRight/egs/audioset/data"
label_csv=os.path.join(local, 'class_labels_indices.csv')
data_path = os.path.join(local,'datafiles/audioset_bal_unbal_train_data.json')

# alpaca_client = llm_client.Client(address="tir-1-7")
# llama_client  = llm_client.Client(address="tir-0-15")

local_client = llm_client.Client(address="localhost", port=5000)

In [ ]:
resp = local_client.prompt("This is a big test of a ")

In [ ]:
resp

In [64]:
data_file = open(data_path)

data = json.load(data_file)


In [59]:
label_dict = {}
with open(label_csv,'r') as data:
    for line in csv.reader(data):
        label_dict[line[1]] = line[2]
        
print(label_dict)

{'mid': 'display_name', '/m/09x0r': 'Speech', '/m/05zppz': 'Male speech, man speaking', '/m/02zsn': 'Female speech, woman speaking', '/m/0ytgt': 'Child speech, kid speaking', '/m/01h8n0': 'Conversation', '/m/02qldy': 'Narration, monologue', '/m/0261r1': 'Babbling', '/m/0brhx': 'Speech synthesizer', '/m/07p6fty': 'Shout', '/m/07q4ntr': 'Bellow', '/m/07rwj3x': 'Whoop', '/m/07sr1lc': 'Yell', '/m/04gy_2': 'Battle cry', '/t/dd00135': 'Children shouting', '/m/03qc9zr': 'Screaming', '/m/02rtxlg': 'Whispering', '/m/01j3sz': 'Laughter', '/t/dd00001': 'Baby laughter', '/m/07r660_': 'Giggle', '/m/07s04w4': 'Snicker', '/m/07sq110': 'Belly laugh', '/m/07rgt08': 'Chuckle, chortle', '/m/0463cq4': 'Crying, sobbing', '/t/dd00002': 'Baby cry, infant cry', '/m/07qz6j3': 'Whimper', '/m/07qw_06': 'Wail, moan', '/m/07plz5l': 'Sigh', '/m/015lz1': 'Singing', '/m/0l14jd': 'Choir', '/m/01swy6': 'Yodeling', '/m/02bk07': 'Chant', '/m/01c194': 'Mantra', '/t/dd00003': 'Male singing', '/t/dd00004': 'Female singing',

In [65]:
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [68]:
out_dict = {}
for i in tqdm(range(len(data["data"]))):
# for i in tqdm(range(5)):
    datum = data["data"][i]
    print(datum)
    labels = datum['labels'].split(",")
    labels = ", ".join([label_dict[label] for label in labels])
    name = datum["wav"].split("wav/")[-1].split(".")[0]
    out_dict[name] = labels

print("Done")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 28036.79it/s]

{'wav': '/home/junchenl/data_folder/AudioSet/unbalanced_wav/---1_cCGK4M.wav', 'labels': '/m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c'}
{'wav': '/home/junchenl/data_folder/AudioSet/unbalanced_wav/---2_BBVHAA.wav', 'labels': '/m/09x0r'}
{'wav': '/home/junchenl/data_folder/AudioSet/unbalanced_wav/---B_v8ZoBY.wav', 'labels': '/m/04rlf'}
{'wav': '/home/junchenl/data_folder/AudioSet/unbalanced_wav/---EDNidJUA.wav', 'labels': '/m/02qldy,/m/02zsn,/m/05zppz,/m/09x0r'}
{'wav': '/home/junchenl/data_folder/AudioSet/unbalanced_wav/---N4cFAE1A.wav', 'labels': '/m/04rlf,/m/09x0r'}
Done


In [64]:
out_file = open("ids_with_captions.json", "w")
json.dump({"data": out_dict}, out_file)

In [5]:
test_file = open("ids_with_captions.json", "r")


test_data = json.load(test_file)["data"]

print(os.chdir("/home/junchenl/diffusers_with_dataloader/examples/text_to_image/audio_gen_files"))



None


In [55]:
top = len(test_data)
n_samples = 50

rand_ids = [random.randint(0, top) for _ in range(n_samples)]

name_list = list(test_data.keys())

sample = [test_data[name_list[idx]] for idx in rand_ids]
ids = [name_list[idx] for idx in rand_ids]
print(sample)

['Music, Dubstep', 'Horse, Clip-clop, Speech, Animal', 'Vehicle, Speech', 'Music', 'Music, Music of Bollywood', 'Speech', 'Music, Speech', 'Speech, Animal', 'Speech, Outside, urban or manmade', 'Speech', 'Music, Rock and roll, Speech', 'Music', 'Snake, Speech', 'Speech', 'Music, Television', 'Boat, Water vehicle, Speech', 'Music, Speech, Male singing', 'Music', 'Music', 'Speech, Child speech, kid speaking', 'Music', 'Boat, Water vehicle, Vehicle', 'Brass instrument, Music, Musical instrument, Orchestra, Trumpet, Classical music', 'Singing, Music, Outside, urban or manmade', 'Music, Speech, Inside, small room', 'Video game music', 'Vehicle, Fixed-wing aircraft, airplane, Aircraft, Outside, urban or manmade', 'Speech', 'Speech', 'Music', 'Music', 'Music, Speech', 'Speech', 'Chainsaw, Power tool', 'Electronic music, Music, Synthesizer', 'Bird, Bird vocalization, bird call, bird song, Chirp, tweet, Coo, Pigeon, dove', 'Music', 'Telephone, Speech', 'Music', 'Music', 'Speech', 'Aircraft', 'E

In [ ]:

def gen_caption(classes, client):
    human_sample = ", ".join(ls_sam[:-2] + [", and ".join(ls_sam[-2:])])
    
    prompt += "For each of the 3 line summarize the sounds into a single sentence: \n"
    prompt += "imagine a scene with all of these sounds existing together: (inside, small room)." + "\n > This is happening inside a small room. \n\n"
    prompt += "imagine a scene with all of these sounds existing together: (dog, bark, howl, speech)." + "\n > a dog barking and howling with a person speaking aswell. \n\n"
    prompt += "imagine a scene with all of these sounds existing together: " + f"({human_sample}). \n >"
    outputs = client.prompt(prompt)
    outputs = outputs[0].text[len(prompt) + 1:]
    final_output = outputs.split("\n")[0]
    return final_output

In [56]:

for sam, idx in zip(sample, ids):
    prompt = ""
    ls_sam  = sam.split(", ")
    human_sample = ", ".join(ls_sam[:-2] + [", and ".join(ls_sam[-2:])])
    
    # prompt =  f"rephrase as sentence: \n"
    # prompt += f"inside, small room => This is happening inside a small room. \n"
    # prompt += f"dog, bark, howl, speech => a dog barking and howling with a person speaking aswell. \n"
    # prompt += f"jazz, music, piano, speech, singing => a musician plays jazz piano while a singer sings. \n"
    # prompt += f"{human_sample} => "
    prompt += "For each of the 3 line summarize the sounds into a single sentence: \n"
    prompt += "imagine a scene with all of these sounds existing together: (inside, small room)." + "\n > This is happening inside a small room. \n\n"
    prompt += "imagine a scene with all of these sounds existing together: (dog, bark, howl, speech)." + "\n > a dog barking and howling with a person speaking aswell. \n\n"
    prompt += "imagine a scene with all of these sounds existing together: " + f"({human_sample}). \n >"
    print("ID: ", idx)
    print("PROMPT: ", f"({human_sample})") 
    # llama_outputs = llama_client.prompt(prompt)
    alpaca_outputs = alpaca_client.prompt(prompt)
    alpaca_outputs = alpaca_outputs[0].text[len(prompt) + 1:]
    final_output = alpaca_outputs.split("\n")[0]
    
    skipped_lines = len(alpaca_outputs.split("\n")) - 1

    # outputs = alpaca_client.prompt(prompt)
    # print(llama_outputs[0].text[len(prompt) + 1:])
    # print("----------------------------------------")
    print("Answer: ", final_output)
    print("\n=================================================\n")

ID:  L7VrvbnX_FU
PROMPT:  (Music, and Dubstep)
Answer:   Music and Dubstep playing as one enters a room.


ID:  1EqtrXZjiJY
PROMPT:  (Horse, Clip-clop, Speech, and Animal)
Answer:   a Horse clip-clopping, a person speaking and an animal (whispering, barking, meow- ing) all existing together.


ID:  wHKWJBlW3Ts
PROMPT:  (Vehicle, and Speech)
Answer:   A vehicle driving and a person speaking aswell.


ID:  F_zkJC5g7zI
PROMPT:  (Music)
Answer:   a violin playing softly in the background, accompanied by a piano and a cello. 


ID:  _obRwk1W0dI
PROMPT:  (Music, and Music of Bollywood)
Answer:   A background of Bollywood music playing while other instruments join in.


ID:  aZY1_FQlPy8
PROMPT:  (Speech)
Answer:   someone speaking quietly.


ID:  kYQc2IjQqRU
PROMPT:  (Music, and Speech)
Answer:   A scene where a person is talking and plays music in the background.


ID:  2vGR5dBBRY0
PROMPT:  (Speech, and Animal)
Answer:   a person speaking and a dog barking and howling.


ID:  KhT2RBSsnJQ
PRO